# ASCAT L1B 数据预处理

## 导入库

In [2]:
import netCDF4
import numpy as np
from pyresample import AreaDefinition, kd_tree, SwathDefinition

## 读数据

In [3]:
l1b_file=r"C:\Datasets\RS\MetopB-ASCAT-1B-12.5KM\2022-06--12\W_XX-EUMETSAT-Darmstadt,SURFACE+SATELLITE,METOPB+ASCAT_C_EUMP_20220601021500_50341_eps_o_125_l1.nc"

l1b_obj = netCDF4.Dataset(l1b_file)
# 关闭netCDF4对数据的自动掩膜和缩放
l1b_obj.set_auto_mask(False)
l1b_obj.set_auto_scale(False)

lon = l1b_obj.variables["lon"][:] * 1e-6
lat = l1b_obj.variables["lat"][:] * 1e-6
sigma0_trip = l1b_obj.variables["sigma0_trip"][:] * 1e-6

l1b_obj.close()

## 数据掩膜

1. 掩膜掉`sigma0`填充值，-2147483648，经过缩放后是-2147.483648，直接用小于-100判断了


In [4]:
mask_conditions = sigma0_trip < -100
sigma0_trip_masked = np.where(mask_conditions, np.nan, sigma0_trip)

## 数据重投影

重投影到0.125°间隔的等经纬度网格

In [5]:
swath = SwathDefinition(lon, lat)
grid = AreaDefinition.from_epsg(4326, 0.125)
sigma0_trip_grid = kd_tree.resample_nearest(swath, sigma0_trip_masked, grid, radius_of_influence=12500, fill_value=np.nan)

## 输出到新的nc文件

写NC文件的套路

1. 新建维度
2. 新建变量
3. 写入变量值

In [6]:
grid_lon, grid_lat = grid.get_lonlats()

output_obj = netCDF4.Dataset("new.nc", "w")

output_obj.createDimension("lon", grid_lon.shape[1])
output_obj.createDimension("lat", grid_lat.shape[0])

output_lon = output_obj.createVariable("lon", "f4", ("lon",))
output_lat = output_obj.createVariable("lat", "f4", ("lat",))
output_sm_fore = output_obj.createVariable("sm_fore", "f4", ("lat","lon",), fill_value=np.nan, compression='zlib')
output_sm_mid = output_obj.createVariable("sm_mid", "f4", ("lat","lon",), fill_value=np.nan, compression='zlib')
output_sm_aft = output_obj.createVariable("sm_aft", "f4", ("lat","lon",), fill_value=np.nan, compression='zlib')

output_sm_fore[:] = sigma0_trip_grid[:, :, 0]
output_sm_mid[:] = sigma0_trip_grid[:, :, 1]
output_sm_aft[:] = sigma0_trip_grid[:, :, 2]
output_lon[:] = grid_lon[0, :]
output_lat[:] = grid_lat[:, 0]

output_obj.close()